#  Import packages

In [ ]:
import numpy as np
import h5py
import glob
import re
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.stats import pearsonr

import sys
sys.path.append('./..')
from src.training_utils import data_load, extract_floats, split_dataset, predict_multi_by_name, plot_violin_and_statistics

from tensorflow import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,BatchNormalization,AveragePooling2D,LeakyReLU,GlobalAveragePooling2D,ReLU

from cmcrameri import cm
import seaborn as sns
import pandas as pd

np.set_printoptions(precision=3, suppress=True)

# Set seed (optional)

In [ ]:
fixed_seed = 216 #choose seed (comment out if not needed)

if 'fixed_seed' in locals():
    keras.utils.set_random_seed(fixed_seed)
    print("Running program with fixed seed:",fixed_seed)
else:
    print("Running program with random seed.")


# Setup GPU

First, follow instructions [here](https://gist.github.com/zrruziev/b93e1292bf2ee39284f834ec7397ee9f), or alternatively run:
```bash
for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done
```
We do this as a workaround for [this error](https://github.com/tensorflow/tensorflow/issues/42738):

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
print(tf.config.list_physical_devices('GPU'), tf.test.gpu_device_name())
print("TF Version:",tf.__version__)

# Define Functions

In [ ]:
def model_predictor (model,x_val,y_val):
    prediction = model.predict(x_val, batch_size=64)
    print("Shape of prediction : ", np.shape(prediction))

    plt.plot(y_val, prediction.T[0], 'o', c='k', alpha=0.25)
    plt.plot(y_val, y_val, 'o', color='r')

    print("Pearson's correlation coeff: ", pearsonr(y_val, prediction.T[0]).statistic)
    plt.xlabel("Input turning rate")
    plt.ylabel("Predicted turning rate")
    plt.axis("equal")
    plt.xscale("log")
    plt.yscale("log")
    return prediction

def violin_plotter (v,y_val,adjustment,legloc="upper left"):
    bins = np.logspace(-6,-1,10, base=2)*0.85

    #v = prediction2.T[0]

    colors = cm.batlowS(np.digitize(v, bins))
    colors_actual = cm.batlowS(np.digitize(np.unique(y_val),bins))

    fig, (ax1,ax2) = plt.subplots(nrows=2,ncols=1,figsize=(9,6),dpi=600)

    df = pd.DataFrame()
    df.insert(0, "predicted", v - y_val)
    df.insert(1, "actual", y_val)

    sns.violinplot(
        ax=ax1,
        data=df,
        x="actual",
        y="predicted",
        color="w",
        alpha=0.7,
        density_norm="width",
        linewidth=1,
        inner="box",
        inner_kws={"box_width": 4, "color": "0.2"},
        )

    ax1.set_xlabel("Actual turning rate")
    ax1.set_ylabel(r"Prediction Difference $P_{pred}-P_{true}$")

    std = []
    means = []
    overlap = []
    std_div = []
    accuracy = 5e-3
    print ("Prediction means and standard deviations.")
    for val in np.unique(y_val):
        v_mapped = v[np.where(y_val == val)]
        stdev = np.std(v_mapped)
        std.append(stdev)
        mean = np.mean(v_mapped)
        overlap.append((val + accuracy >= np.min(v_mapped)) & (val - accuracy <= np.max(v_mapped)))
        within_std = abs(val-mean)/stdev
        print (f"Actual value {val}: Average = {mean:.5f} +- {stdev:.5f}; Expected value within {within_std:.3f} stdevs of mean")
        std_div.append(within_std)

    print(f"With accuracy {accuracy}, overlap ratio:", np.sum(overlap)/len(overlap))
    print("(Min, Max, Avg) STD:", np.min(std), np.max(std), np.mean(std))
    print("Pearson's correlation coeff: ", pearsonr(y_val, v).statistic)



    for val in np.unique(y_val):
        v_mapped = v[np.where(y_val == val)]
        means.append(np.mean(v_mapped))

    ax2.errorbar(np.sort(np.unique(y_val)),np.abs(means-np.sort(np.unique(y_val))),yerr=(std),ecolor='black',elinewidth=0.5,capsize=3,color='purple',label=r'$|\langle P_{pred} \rangle -P_{true}|$')
    ax2.plot(np.sort(np.unique(y_val)),np.zeros(np.unique(y_val).shape[0]),color='red',label='True value line',linestyle='dotted',alpha=0.5)


    ax2.legend(loc=legloc)

    counter = 0
    for i in np.sort(np.unique(y_val)):
        ax2.text(i,adjustment,f"${std_div[counter]:.3f} \sigma$",ha="center")
        counter = counter + 1

    ax2.set_xscale("log")
    ax2.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
    ax2.set_xticks(np.unique(y_val))

    ax2.set_xlabel("Actual turning rate")
    ax2.set_ylabel("Absolute mean prediction difference")

    fig.tight_layout()

def mae_plotter(history,epochs,legloc='upper right'):
    acc = history.history['mae']
    val_acc = history.history['val_mae']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.plot(epochs_range, acc, label='Training MAE')
    plt.plot(epochs_range, val_acc, label='Validation MAE')
    plt.legend(loc=legloc)
    plt.title('Training and Validation MAE')

# Import and prepare data

set model1 to have orientation, model2 to be monochrome

In [ ]:
#all alphas: [0.016,0.023,0.034,0.050,0.073,0.107,0.157,0.231,0.340,0.500]
#all densities: [0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95]
x1,y,shape = data_load(alphas=[0.016,0.023,0.034,0.050,0.073,0.107,0.157,0.231,0.340,0.500], densities=[0.25],orientation=True,scrambled=False)
x2 = np.ceil(x1)

We have N * number of unique alpha snapshots total, we split them into training set and a validation set with the ratio 80/20:

In [ ]:
print("Orientation model:")
x_train1, y_train, x_val1, y_val = split_dataset(x1,y,last=int(len(x1)*0.2)) #len(x)*1 means no training, only validation!
x_train2 = np.ceil(x_train1)
x_val2 = np.ceil(x_val1)
#print("\nMonochrome model:")
#x_train2, y_train2, x_val2, y_val = split_dataset(x2,y2,last=int(len(x2)*0.2)) #len(x)*1 means no training, only validation!

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2)
ax1.matshow(x_val1[500],cmap=plt.get_cmap(name="gnuplot",lut=5))
ax2.matshow(x_val2[500],cmap=plt.get_cmap(name="gnuplot",lut=2))

# Setup and train our models

Workaround for dropout not working:

In [ ]:
import contextlib

@contextlib.contextmanager
def options(options):
  old_opts = tf.config.optimizer.get_experimental_options()
  tf.config.optimizer.set_experimental_options(options)
  try:
    yield
  finally:
    tf.config.optimizer.set_experimental_options(old_opts)

## Setting up the models' architecture

In [ ]:
def make_net(shape):
    model = Sequential()

    model.add(Conv2D(filters=3, kernel_size=(3, 3), padding="same", input_shape=shape))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(ReLU())
    model.add(BatchNormalization())

    model.add(Conv2D(filters=4, kernel_size=(5, 5), padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(ReLU())
    model.add(BatchNormalization())

    model.add(Conv2D(filters=6, kernel_size=(5, 5), padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(ReLU())
    model.add(BatchNormalization())

    model.add(GlobalAveragePooling2D())

    with options({"layout_optimizer": False}):
        model.add(Dropout(0.1))

    model.add(Dense(units=128, activation="relu"))

    with options({"layout_optimizer": False}):
        model.add(Dropout(0.1))

    model.add(Dense(units=3, activation="relu"))

    model.add(Flatten())
    model.add(Dense(units=1, activation="linear"))
    return model

# MODEL 1 (ORIENTATION)

In [ ]:
model1 = make_net(shape)

## Optimizer

In [ ]:
#optimizer = keras.optimizers.Adam(learning_rate=0.0003)
optimizer = keras.optimizers.SGD(learning_rate=0.005)
model1.compile(loss='mae', optimizer=optimizer, metrics=['mae'])
model1.summary()

## Training and evaluation

Before training, these are the "predictions":

In [ ]:
prediction1 = model_predictor(model1,x_val1,y_val)

In [ ]:
demo_idx = 100
plt.matshow(x_val1[demo_idx])
print("Actual: ", y_val[demo_idx])
print("Predicted: ", prediction1.T[0][demo_idx])

We can play with the architecture and see how the untrained predictions can change too.

## Run the training

### model1

In [ ]:
epochs=60
batch_size=64
history1 = model1.fit(
    x_train1,
    y_train,
    epochs=epochs,
    verbose=True,
    batch_size=batch_size,
    validation_data=(x_val1, y_val)
)

# MODEL 2 (MONOCHROME)

In [ ]:
model2 = make_net(shape)

# Optimizer

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.005)
model2.compile(loss='mae', optimizer=optimizer, metrics=['mae'])
model2.summary()

# Training and Evaluation

In [ ]:
prediction2 = model_predictor(model2,x_val2,y_val)

In [ ]:
plt.matshow(x_val1[demo_idx])
print("Actual: ", y_val[demo_idx])
print("Predicted: ", prediction1.T[0][demo_idx])

## model2

In [ ]:
epochs=60
batch_size=64
history2 = model2.fit(
    x_train2,
    y_train,
    epochs=epochs,
    verbose=True,
    batch_size=batch_size,
    validation_data=(x_val2, y_val)
)

In [ ]:
print("Evaluate on test data:")
results1 = model1.evaluate(x_val1, y_val, batch_size=batch_size, verbose=0)
print("Test loss for model1 (orientation):", results1)
results2 = model2.evaluate(x_val2, y_val, batch_size=batch_size, verbose=0)



# Analyse training results

In [ ]:
prediction1 = model1.predict(x_val1)
print("Shape of prediction1 (model1 on x_val1) : ", np.shape(prediction1))
prediction12 = model1.predict(x_val2)
print("Shape of prediction12 (model1 on x_val2) : ", np.shape(prediction12))
prediction2 = model2.predict(x_val2)
print("Shape of prediction2 (model2 on x_val2) : ", np.shape(prediction2))
prediction21 = model2.predict(x_val1)
print("Shape of prediction21 (model2 on x_val1) : ", np.shape(prediction21))

# Plot model1

## Predict on xval1

In [ ]:
violin_plotter(prediction1.T[0],y_val,-0.02)

## predict on xval2

In [ ]:
violin_plotter(prediction12.T[0],y_val,0.01)

## MAE

In [ ]:
mae_plotter(history1,epochs)

# Plot model2

## on x_val2

In [ ]:
violin_plotter(prediction2.T[0],y_val,-0.02)

## on x_val1

In [ ]:
violin_plotter(prediction21.T[0],y_val,0.07,'upper right')

In [ ]:
mae_plotter(history2,epochs)

# Save models (if needed)

In [ ]:
name1 = "daedalus5812" #ORIENTATION MODEL
model1.save(f"../models/{name1}.keras")
name2 = "icarus1508" #MONOCHROME MODEL
model2.save(f"../models/{name2}.keras")

print("Name for combined .html file: "+name1+"-"+name2)